# Producer

In [1]:
import kafka
from kafka       import KafkaProducer
from kafka.admin import KafkaAdminClient
from kafka.admin import NewTopic

from tqdm        import tqdm

import pandas as pd
import os
import boto3
import json
import time

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
KAFKA_HOME = '/usr/local/kafka'
KAFKA_BOOTSTRAP_SERVERS = ['localhost:9092']

In [3]:
# producer definition from IP address given before
producer = KafkaProducer(bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS)

# connect to the cluster to run admin functions
kafka_admin = KafkaAdminClient(
        bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
        api_version=(0,10,2))

In [4]:
kafka_admin.list_topics()

['stream', 'topic_stream', 'results', '__consumer_offsets']

In [6]:
kafka_admin.delete_topics(['stream', 'topic_stream'])

UnknownTopicOrPartitionError: [Error 3] UnknownTopicOrPartitionError: Request 'DeleteTopicsRequest_v3(topics=['stream', 'topic_stream'], timeout=30000)' failed with response 'DeleteTopicsResponse_v3(throttle_time_ms=0, topic_error_codes=[(topic='stream', error_code=3), (topic='topic_stream', error_code=0)])'.

In [8]:
stream = NewTopic(name='stream', 
                       num_partitions=1, 
                       replication_factor=1)

kafka_admin.create_topics(new_topics=[stream])

TopicAlreadyExistsError: [Error 36] TopicAlreadyExistsError: Request 'CreateTopicsRequest_v3(create_topic_requests=[(topic='stream', num_partitions=1, replication_factor=1, replica_assignment=[], configs=[])], timeout=30000, validate_only=False)' failed with response 'CreateTopicsResponse_v3(throttle_time_ms=0, topic_errors=[(topic='stream', error_code=36, error_message="Topic 'stream' already exists.")])'.

In [9]:
kafka_admin.list_topics()

['stream', 'results', '__consumer_offsets']

In [10]:
# load data from s3 bucket
# the keys are useless as verify is false
s3_client = boto3.client('s3',
                         endpoint_url='https://cloud-areapd.pd.infn.it:5210',
                         aws_access_key_id='1a4543841b844a88bb3f2eba45764d61',
                         aws_secret_access_key='42e2f16592f54668b8421ecf5ca7ba51',
                         verify=False)

list_obj_contents = s3_client.list_objects(Bucket= 'mapd-minidt-stream')['Contents']

# wait time to have ~1000 rows/s
wait_time = .00018

for i in range(0, len(list_obj_contents)):
    file_name = list_obj_contents[i]['Key']
    row_data = pd.read_csv(s3_client.get_object(Bucket='mapd-minidt-stream', 
                                                Key=file_name).get('Body'), dtype='str')
    # convert file to json
    json_data = row_data.to_dict( 'records' )
    
    # tqdm to visualize progresses
    for msg in tqdm(json_data): 
        producer.send('topic_stream', json.dumps(msg).encode('utf-8'))
        time.sleep(wait_time)
    producer.flush()


 62%|████████████████████████████▌                 | 812832/1310720 [10:35<06:29, 1279.87it/s]

KeyboardInterrupt

